In [1]:
from sklearn.decomposition import NMF
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV, GridSearchCV
from sklearn.metrics import silhouette_score
from sklearn.feature_extraction.text import TfidfVectorizer

# Define custom scoring function
def my_score(X, labels):
    return silhouette_score(X, labels=True)

from sklearn.metrics import mean_squared_error, mean_squared_log_error

!pip install pandas==1.5.3
import pandas as pd

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:

from google.colab import drive

drive.mount('/content/gdrive', force_remount=True)
%cd gdrive/MyDrive/TxtAnalytics_Proj

searchMatrix = pd.read_pickle("X_Train_nonSuicidal.pkl")

Mounted at /content/gdrive
/content/gdrive/MyDrive/TxtAnalytics_Proj


In [ ]:
def delete_less_than_k(arr,k):
  temp = [] #Temporary list
  for val in arr:#Iterating the list arr
    if len(val)<=k:
      temp.append(val)
  #All the elements whose value is less than k are stored in temp
  for i in temp:#Iterating the temporary list
    arr.remove(i)
  
  return arr



In [3]:

stop_list = ("suicidal", "suicide", "aah", "fuck", "fucking", "na")
text_data = searchMatrix['text_lemmatized'].to_numpy()
text_data = [[w for w in doc if w not in stop_list] for doc in text_data]

text_data_lemmatized_joined = [' '.join(x) for x in text_data]  # joined to fit CountVectorizer
print(text_data_lemmatized_joined[:6])
#text_data_lemmatized_joined = delete_less_than_k(text_data_lemmatized_joined, 3)

vectorizer = TfidfVectorizer(stop_words='english')
# Fit the vectorizer and transform the text data
data_vectorized = vectorizer.fit_transform(text_data_lemmatized_joined)

#feature_names1 = vectorizer.get_feature_names_out()

['go adult year crazy friend viking feel birthday day earth like constant countdown july head know feel like big deal maybe cause parent let live date id able want approval surreal have hold have thing new dentist new doctor new chapter fact childhood think keep childhood make break hard try fix hard forgive people stop kid feel way stress college stuff right sleep', 'today birthday today birthday generally person do nt forget stuff easy forget someone birthday large friend group know birthdays heart know wish happy birthday day today wake get text sex girlfriend complicate get mental issue decide consider year abroad soon wish happy birthday happy sad time big deal friend birthdays gift outside wifi shuts cell reception wish happy birthday right day begin help fairly large streamer help best friend work present girlfriend want easy believe people forget pretty surprised talk good friend today live talk day past week school didst remember talk yesterday si room ask cake id feel like pe

In [4]:

#NMF_model2 = NMF(alpha_H= 0.001, 
#                alpha_W= 0, 
#                l1_ratio=0.2, 
#                n_components=10
#                ,init = 'nndsvd'
#                )

NMF_model2 = NMF(alpha_H= 0.001, 
                 random_state = 42,
                 alpha_W= 0, 
                 l1_ratio=0.1, 
                 n_components=10,
                 max_iter = 5000
                ,init = 'nndsvd'
                
                )
nmf_output2 = NMF_model2.fit_transform(data_vectorized)
print(nmf_output2)  # Model attributes

#print(data_vectorized[:20][0:])
# Print the top words for each topic
feature_names = vectorizer.get_feature_names_out()
#for topic_idx, topic in enumerate(NMF_model.components_):
#    top_words = [feature_names[i] for i in topic.argsort()[:-15 - 1:-1]]
#    print(f"Topic {topic_idx}: {', '.join(top_words)}")

for topic_idx, topic in enumerate(NMF_model2.components_):
    top_words = [feature_names[i] for i in topic.argsort()[:-15 - 1:-1]]
    print(f"Topic {topic_idx}: {', '.join(top_words)}")



[[3.34303542e+02 0.00000000e+00 0.00000000e+00 ... 3.23969687e+01
  2.58149481e+01 2.03156523e+01]
 [4.25233635e+02 3.90920475e-02 3.06226323e+01 ... 4.41189870e+01
  0.00000000e+00 3.97307443e+01]
 [5.49908525e+01 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  8.00456406e+00 3.88919074e+00]
 ...
 [1.38239245e+02 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  2.16222518e+00 0.00000000e+00]
 [6.97396137e+01 1.67454024e-02 4.63182493e+00 ... 0.00000000e+00
  4.81712983e+01 4.02043010e+00]
 [2.63395156e+02 0.00000000e+00 0.00000000e+00 ... 5.60876684e+00
  0.00000000e+00 8.93413297e+00]]
Topic 0: friend, know, school, think, time, year, live, say, tell, people, thing, start, life, didst, bad
Topic 1: filler, text, ask, title, bore, miller, filer, award, id, low, code, whats, mod, lao, gay
Topic 2: talk, wan, bore, chat, pm, id, hmm, lonely, people, kinda, discord, bored, friend, anybody, snap
Topic 3: post, comment, award, geddit, shit, sub, people, karma, text, edit, meme, image,

/usr/local/lib/python3.9/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 5000 reached. Increase it to improve convergence.
  warnings.warn(


In [5]:
import joblib
import numpy as np

#NMF_model2 = joblib.load('suicidal_nmf_model2.jl')
#nmf_output2 = joblib.load('suicidal_nmf_output2.jl')

W = nmf_output2
H = NMF_model2.components_
# compute log likelihood
n_samples, n_features = data_vectorized.shape
reconstruction_err = NMF_model2.reconstruction_err_
log_likelihood = -0.5 * reconstruction_err / n_samples - 0.5 * n_features * np.log(2 * np.pi)


print("Log likelihood:", log_likelihood)

#perplexity = exp(-log likelihood / total number of words)
perplexity = np.exp(-log_likelihood / data_vectorized.sum())

# Perplexity: Lower the better. Perplexity = exp(-1. * log-likelihood per word)
print("Perplexity: ", perplexity)
# See model parameters
print(NMF_model2.get_params())

Log likelihood: -31685.002652427007
Perplexity:  1.1761322832169279
{'alpha_H': 0.001, 'alpha_W': 0, 'beta_loss': 'frobenius', 'init': 'nndsvd', 'l1_ratio': 0.1, 'max_iter': 5000, 'n_components': 10, 'random_state': 42, 'shuffle': False, 'solver': 'cd', 'tol': 0.0001, 'verbose': 0}


In [6]:
import joblib

# Save a model to disk
joblib.dump(NMF_model2, 'nonsuicidal_nmf_model1.jl')
joblib.dump(nmf_output2, 'nonsuicidal_nmf_output1.jl')

['nonsuicidal_nmf_output1.jl']

In [7]:

import numpy as np

epsilon = 1e-8
W_norm = W / (W.sum(axis=1, keepdims=True) + epsilon)
# find the rows in W_norm that do not sum up to 1
problematic_rows = np.where(np.abs(W_norm.sum(axis=1) - 1) > epsilon)[0]
W_norm[problematic_rows, :] = 1
# print the indices of the problematic rows
print("Problematic rows:", len(problematic_rows))

Problematic rows: 165


In [8]:
!pip install pyldavis pyLDAvis==3.4.0
import pyLDAvis
import pyLDAvis.lda_model
pyLDAvis.enable_notebook()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 33.8 MB/s eta 0:00:00


In [11]:
print('The pyLDAvis-learn version is {}.'.format(pyLDAvis.__version__))
panel = pyLDAvis.lda_model.prepare(NMF_model2, data_vectorized, vectorizer, 
                                   mds='tsne'
                                   #,topic_term_dists=H_norm.T
                                   ,doc_topic_dists=W_norm
)
panel

The pyLDAvis-learn version is 3.4.0.


/usr/local/lib/python3.9/dist-packages/pyLDAvis/lda_model.py:26: RuntimeWarning: invalid value encountered in true_divide
  return dists / dists.sum(axis=1)[:, None]
/usr/local/lib/python3.9/dist-packages/pandas/core/internals/blocks.py:351: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)
/usr/local/lib/python3.9/dist-packages/pyLDAvis/_prepare.py:243: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/usr/local/lib/python3.9/dist-packages/pandas/core/internals/blocks.py:351: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


PreparedData(topic_coordinates=               x          y  topics  cluster       Freq
topic                                                  
0      37.461754  30.320192       1        1  46.121832
4     -47.884182  37.249905       2        1   9.601647
8     -29.345371  -7.610730       3        1   8.858487
5     -70.065834  -6.906518       4        1   7.046987
3      50.974426 -16.126734       5        1   7.001470
9      -5.417380  24.274990       6        1   6.848411
6      10.186985 -12.471421       7        1   5.982810
7     -35.264175 -48.922451       8        1   4.322225
2      11.650851 -53.922260       9        1   3.606669
1      -3.258632  64.655724      10        1   0.609461, topic_info=          Term         Freq        Total Category  logprob  loglift
33247     want  1751.000000  1751.000000  Default  30.0000  30.0000
10243   filler   937.000000   937.000000  Default  29.0000  29.0000
6899       day  1573.000000  1573.000000  Default  28.0000  28.0000
18176     like  2018.000000  2018.000000  Default  27.0000  27.0000
23511     post  1562.000000  1562.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
6052      cool     1.347290   235.098196  Topic10  -6.7903  -0.0616
13648    horny     1.233546   202.934467  Topic10  -6.8785  -0.0026
17049     join     1.242177   268.799357  Topic10  -6.8715  -0.2767
23254     plus     1.118852   182.411657  Topic10  -6.9760   0.0064
5525   comment     1.112376   505.946595  Topic10  -6.9819  -1.0196

[1050 rows x 6 columns], token_table=       Topic      Freq                                               Term
term                                                                     
3          2  1.118188                                         aaaaaaaaaa
22         4  1.651337                  aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
27         4  1.168477     aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa
37         6  0.839677  aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa...
57         7  1.451946           aaaaaaaaaaaaaaaaaaaaaaqaaaaaaaaaaaaaaaaa
...      ...       ...                                                ...
34338      3  0.438412                                            youtube
34338      5  0.098152                                            youtube
34338      6  0.215934                                            youtube
34338      8  0.019630                                            youtube
34352      5  1.238364                            youyesgaythxhineutralok

[1990 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 5, 9, 6, 4, 10, 7, 8, 3, 2])

In [12]:
pyLDAvis.save_html(panel, 'NMF_Nonsuicidal.html')